In [ ]:
import intake
import os
import zarr

In [ ]:
col_url = (
    "https://cmip6downscaling.blob.core.windows.net/cmip6/pangeo-cmip6.json"
)

col = intake.open_esm_datastore(col_url)

full_subset = col.search(
    activity_id=["CMIP", "ScenarioMIP"],
    experiment_id=["historical"],  # , "ssp245", "ssp370", "ssp585"
    member_id="r1i1p1f1",
    source_id="BCC-CSM2-MR",
    table_id="day",
    grid_label="gn",
    variable_id=["tasmax"],  # tasmax, tasmin, pr
)

In [ ]:
def get_store(bucket, prefix, account_key=None):
    """helper function to create a zarr store"""

    if account_key is None:
        account_key = os.environ.get("AccountKey", None)

    store = zarr.storage.ABSStore(
        bucket,
        prefix=prefix,
        account_name="cmip6downscaling",
        account_key=account_key,
    )
    return store

In [ ]:
full_subset.df["zstore"].item()

In [ ]:
get_store(
    bucket="cmip6",
    prefix="CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/day/pr/gn/v20181126/",
)

In [ ]:
ds = full_subset.to_dataset_dict(
    zarr_kwargs={
        "consolidated": True,
        "decode_times": True,
        "use_cftime": True,
    },
    storage_options={
        "account_name": "cmip6downscaling",
        "account_key": os.environ.get("AccountKey", None),
    },
)

In [ ]:
ds['CMIP.BCC.BCC-CSM2-MR.historical.day.gn'].tasmax.isel(time=slice(0,30)).

In [ ]:
ds["CMIP.BCC.BCC-CSM2-MR.historical.day.gn"]

In [ ]:
def downscale(method_specs):
    downscale_model = get_model(method_specs, model_params=method_specs)
    downscale_model.prep()  # calls regrid
    downscale_model.fit()  # quantile mapping
    downscale_model.predict()
    downscale_model.eval()

In [ ]:
def get_model():
    ensure_consistency(
        method_specs, obs
    )  # either check that params match obs or that obs can be coerced into matching specs
    return quantile_mapping  # this will be a class

In [ ]:
def quantile_mapping(gcm, obs)

In [ ]:
# grab a gcm
# grab training data
downscale(method_specs={'name': 'quantile_mapping',  # when you regrid gets decided by the method 'chiao'
#                         'varibles':
#                         'spatial_resolution': ,
#                         'temporal_resolution':}, 
                        # which choices do we want to expose and test/run multiple permutations of
          gcm=gcm, 
          scenario=scenario, 
          region='global', 
         )
evaluate_performance() 
# gcm changes and metrics AND obs ;
# how well is it actually bias-corrected- what biases remain?
evaluate_changes() # store the change signal? 

thoughts:

- when to take from raw gcm grid to output - depends on method (maybe a
  downscaling method will keep it on its raw grid and find nearest neighbor or
  patterns?)
- what about other cmip6 experiments (not just scenario mip)
